In [1]:
# Dependencies for this notebook can be found in requirements_autogluon.txt

# Install dependencies using pip3 install -r requirements_autogluon.txt

# Installation instructions

# virtualenv venv_autogluon
# source venv_autogluon/bin/activate
# ipython kernel install --user --name=venv_autogluon
# Start the notebook using `python3 -m jupyter notebook`
# In the notebook click on Kernel -> Change kernel -> venv_autogluon
# Execute this notebook

# Uncomment these lines if not using a kernal specifically for autogluon

# import sys
# print(sys.executable) 

# ! /usr/local/opt/python@3.9/bin/python3.9 -m pip install autogluon

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('data/UNSW_NB15_training-set.csv')

In [4]:
df_test = pd.read_csv('data/UNSW_NB15_testing-set.csv')

In [5]:
df = df.drop(['attack_cat', 'id'], axis=1, errors='ignore')

In [6]:
df_test = df_test.drop(['attack_cat', 'id'], axis=1, errors='ignore')

In [7]:
X_train = df[[col for col in df.columns if col != 'label']]
y_train = df['label']

In [8]:
X_test = df_test[[col for col in df_test.columns if col != 'label']]
y_test = df_test['label']

In [9]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [11]:
training_set = TabularDataset(df)

In [12]:
subsample_size = 50000
training_set_sample = training_set.sample(n=subsample_size, random_state=0)
training_set_sample.head()

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,label
75179,0.000003,tcp,-,REQ,2,0,90,0,333333.321500,254,...,1,1,6,0,0,0,4,6,0,0
20654,0.000004,udp,dns,INT,2,0,114,0,250000.000600,254,...,18,18,21,0,0,0,19,21,0,1
56735,0.000007,sps,-,INT,2,0,200,0,166666.660800,254,...,6,6,6,0,0,0,9,10,0,1
48735,0.285503,tcp,http,FIN,10,6,876,268,52.538852,254,...,1,1,1,0,0,1,1,2,0,1
54791,2.069207,tcp,ftp-data,FIN,8,8,364,1572,7.249154,62,...,1,1,2,0,0,0,2,1,0,1


In [21]:
##### MAY NEED TO RUN THE FOLLOWING IF YOUR KERNEL DIES IN THE STEP BELOW #####

# Uninstall libomp if it was previous installed
# brew uninstall -f libomp
# wget https://raw.githubusercontent.com/Homebrew/homebrew-core/fb8323f2b170bd4ae97e1bac9bf3e2983af3fdb0/Formula/libomp.rb
# brew install libomp.rb
# rm libomp.rb

In [13]:
save_path = 'models'  # specifies folder to store trained models
label = 'label'
predictor = TabularPredictor(label=label, path=save_path).fit(training_set_sample)

Beginning AutoGluon training ...
AutoGluon will save models to "models/"
AutoGluon Version:  0.7.0
Python Version:     3.9.16
Operating System:   Darwin
Platform Machine:   x86_64
Platform Version:   Darwin Kernel Version 21.4.0: Fri Mar 18 00:45:05 PDT 2022; root:xnu-8020.101.4~15/RELEASE_X86_64
Train Data Rows:    50000
Train Data Columns: 42
Label Column: label
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    8380.83 MB
	Train Data (Original)  Memory Usage: 24.56 MB (0.3% of available memor

[1000]	valid_set's binary_error: 0.0264


	0.9736	 = Validation score   (accuracy)
	3.58s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: LightGBM ...
	0.9816	 = Validation score   (accuracy)
	1.5s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.9772	 = Validation score   (accuracy)
	4.54s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.9756	 = Validation score   (accuracy)
	3.42s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: CatBoost ...
	0.9716	 = Validation score   (accuracy)
	7.02s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.976	 = Validation score   (accuracy)
	1.84s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.9756	 = Validation score   (accuracy)
	1.91s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	0.968	 = Validation score   (accuracy)
	46.45s	 = Training   runtime
	0.04s	 

In [14]:
testing_set = TabularDataset(df_test)

In [15]:
predictor.leaderboard(testing_set, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT,0.902048,0.9736,2.659759,0.040016,3.575618,2.659759,0.040016,3.575618,1,True,3
1,RandomForestEntr,0.901438,0.9756,1.142674,0.064588,3.421957,1.142674,0.064588,3.421957,1,True,6
2,XGBoost,0.901153,0.9808,1.010890,0.027725,3.825059,1.010890,0.027725,3.825059,1,True,11
3,CatBoost,0.900919,0.9716,0.118867,0.009403,7.024489,0.118867,0.009403,7.024489,1,True,7
4,RandomForestGini,0.900731,0.9772,0.984688,0.070347,4.544307,0.984688,0.070347,4.544307,1,True,5
5,ExtraTreesGini,0.899807,0.9760,1.334867,0.076038,1.843429,1.334867,0.076038,1.843429,1,True,8
6,ExtraTreesEntr,0.899402,0.9756,1.179853,0.073673,1.907822,1.179853,0.073673,1.907822,1,True,9
7,LightGBM,0.899350,0.9816,1.236820,0.025009,1.504498,1.236820,0.025009,1.504498,1,True,4
8,LightGBMLarge,0.898649,0.9824,1.306724,0.023212,2.756242,1.306724,0.023212,2.756242,1,True,13
9,WeightedEnsemble_L2,0.898649,0.9824,1.310353,0.026972,3.937611,0.003629,0.003760,1.181369,2,True,14


In [16]:
predictor.info().get('model_info').get('NeuralNetFastAI')

{'name': 'NeuralNetFastAI',
 'model_type': 'NNFastAiTabularModel',
 'problem_type': 'binary',
 'eval_metric': 'accuracy',
 'stopping_metric': 'accuracy',
 'fit_time': 46.451650857925415,
 'num_classes': 2,
 'quantile_levels': None,
 'predict_time': 0.041237831115722656,
 'val_score': 0.968,
 'hyperparameters': {'layers': None,
  'emb_drop': 0.1,
  'ps': 0.1,
  'bs': 'auto',
  'lr': 0.01,
  'epochs': 'auto',
  'early.stopping.min_delta': 0.0001,
  'early.stopping.patience': 20,
  'smoothing': 0.0},
 'hyperparameters_fit': {'epochs': 30, 'best_epoch': 28},
 'hyperparameters_nondefault': [],
 'ag_args_fit': {'max_memory_usage_ratio': 1.0,
  'max_time_limit_ratio': 1.0,
  'max_time_limit': None,
  'min_time_limit': 0,
  'valid_raw_types': ['bool', 'int', 'float', 'category'],
  'valid_special_types': None,
  'ignored_type_group_special': ['text_ngram', 'text_as_category'],
  'ignored_type_group_raw': None,
  'get_features_kwargs': None,
  'get_features_kwargs_extra': None,
  'predict_1_bat

In [17]:
predictor.info().get('model_info').get('NeuralNetTorch')

{'name': 'NeuralNetTorch',
 'model_type': 'TabularNeuralNetTorchModel',
 'problem_type': 'binary',
 'eval_metric': 'accuracy',
 'stopping_metric': 'accuracy',
 'fit_time': 147.87898111343384,
 'num_classes': 2,
 'quantile_levels': None,
 'predict_time': 0.04313969612121582,
 'val_score': 0.9716,
 'hyperparameters': {'num_epochs': 500,
  'epochs_wo_improve': 20,
  'activation': 'relu',
  'embedding_size_factor': 1.0,
  'embed_exponent': 0.56,
  'max_embedding_dim': 100,
  'y_range': None,
  'y_range_extend': 0.05,
  'dropout_prob': 0.1,
  'optimizer': 'adam',
  'learning_rate': 0.0003,
  'weight_decay': 1e-06,
  'proc.embed_min_categories': 4,
  'proc.impute_strategy': 'median',
  'proc.max_category_levels': 100,
  'proc.skew_threshold': 0.99,
  'use_ngram_features': False,
  'num_layers': 4,
  'hidden_size': 128,
  'max_batch_size': 512,
  'use_batchnorm': False,
  'loss_function': 'auto'},
 'hyperparameters_fit': {'batch_size': 128, 'num_epochs': 126},
 'hyperparameters_nondefault': [

In [18]:
hyperparameters = {
   'NN': {'num_epochs': 500},
   'CAT': {'iterations': 10000},
   'RF': {'n_estimators': 300},
   'XT': {'n_estimators': 300},
   'KNN': {},
}

In [19]:
save_path = 'ag_models'  # specifies folder to store trained models
predictor_tuned = TabularPredictor(label=label).fit(training_set_sample, hyperparameters=hyperparameters)

No path specified. Models will be saved in: "AutogluonModels/ag-20230326_224431/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20230326_224431/"
AutoGluon Version:  0.7.0
Python Version:     3.9.16
Operating System:   Darwin
Platform Machine:   x86_64
Platform Version:   Darwin Kernel Version 21.4.0: Fri Mar 18 00:45:05 PDT 2022; root:xnu-8020.101.4~15/RELEASE_X86_64
Train Data Rows:    50000
Train Data Columns: 42
Label Column: label
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available

In [20]:
predictor_tuned.leaderboard(testing_set, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.901443,0.9784,2.212489,0.163861,12.248064,0.020369,0.003731,0.397842,2,True,5
1,CatBoost,0.900919,0.9716,0.112817,0.008040,6.081161,0.112817,0.008040,6.081161,1,True,3
2,RandomForest,0.900731,0.9772,0.904907,0.075692,3.881821,0.904907,0.075692,3.881821,1,True,2
3,ExtraTrees,0.899807,0.9760,1.174397,0.076398,1.887240,1.174397,0.076398,1.887240,1,True,4
4,KNeighbors,0.812445,0.8016,6.524454,0.189028,0.062836,6.524454,0.189028,0.062836,1,True,1
